In [ ]:
from random import random

sentences=[]
f = open("/home/kesci/input/bytedance/train_final.csv")
for line in f:
    line = line.strip().split(",")
    if random()<0.1:
        sentences.append(line[1])
        sentences.append(line[3])
f.close()

f=open("/home/kesci/input/bytedance/bytedance_contest.final_2.csv")
for line in f:
    line = line.strip().split(",")
    sentences.append(line[1])
    sentences.append(line[3])
f.close()

In [ ]:
# 构建word_index
words_index = {}
for sentence in sentences:
    for word in sentence.split():
        words_index.setdefault(word,0)
        words_index[word] += 1

new_words_index = sorted(words_index.items(), key=lambda item:item[1], reverse=True)   # 降序排列

In [ ]:
# word_index存入文件
idx=1
word_count=0
f=open("nn_words_index.txt","w")
for pairs in new_words_index:
    if pairs[1] >= 4:             # 词频数，大于4的都要
        word_count += 1
        f.write(pairs[0]+" "+str(idx)+"\n")
        idx += 1
print(word_count)
f.close()

In [ ]:
# sentence写入文件，用于训练词向量
import time
import os
from gensim.models import Word2Vec
import gensim
import csv

def data_to_sentence():
    if not os.path.exists("nn_sentences"):
        f = open("nn_sentences", "w")
        f.close()

    with open("nn_sentences", "w") as f:
        for s in sentences:
            s = s.replace("\t", "")
            f.write(s + "\n")
    f.close()

    print("Writing sentence Successful!")
data_to_sentence()

In [ ]:
from gensim.models import FastText
import gensim
import time

model = FastText(min_count=4, window=5, size=150, workers=4)
model.build_vocab(corpus_file="nn_sentences")
model.train(corpus_file="nn_sentences", total_words=model.corpus_total_words,
            epochs=5, compute_loss=True)
model.save("fasttext.model")